# Baseline Kaggle - 3º Hackdays CDS

In [1]:
import pandas                as pd
from sklearn.preprocessing   import LabelEncoder
from sklearn                 import model_selection as ms
from xgboost                 import XGBClassifier
from sklearn.metrics         import f1_score
from sklearn.model_selection import cross_val_score
import io 
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import gridspec
from IPython.display import display, HTML
from IPython.display import Image
import seaborn as sns
import csv
from scipy.stats import chi2_contingency
from tabulate import tabulate
from tabulate import tabulate
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.pipeline import Pipeline, make_pipeline
import sklearn
import io 

In [2]:
def jupyter_settings():
    %matplotlib inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [15, 8]
    plt.rcParams['font.size'] = 12
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.set_option( 'display.expand_frame_repr', False )
    pd.options.display.max_columns = None
    sns.set(font_scale=1.5)
jupyter_settings()

In [3]:
# Carregar dataset treino kaggle
from google.colab import files  
uploaded = files.upload()
df_train = pd.read_csv(io.BytesIO(uploaded['train.csv'])) 

Saving train.csv to train.csv


In [4]:
# Carregar dataset teste kaggle
uploaded = files.upload()
df_test = pd.read_csv(io.BytesIO(uploaded['test.csv'])) 

Saving test.csv to test.csv


In [25]:
# Listar dataset
df = df_train.copy()
# df

In [26]:
# df_train.describe().T

df1 = df.copy()

In [27]:
df1['idade'] = np.where(df1['idade'] > 95, 38,df1['idade'] )
df1['num_emprestimos'] = np.where(df1['num_emprestimos'] > 20, 3,df1['num_emprestimos'] )
df1['num_contas_bancarias'] = np.where(df1['num_contas_bancarias'] > 12, 6,df1['num_contas_bancarias'] )
df1['num_cartoes_credito'] = np.where(df1['num_cartoes_credito'] > 12, 5,df1['num_cartoes_credito'] )
df1['num_pgtos_atrasados'] = np.where(df1['num_pgtos_atrasados'] > 24, 14,df1['num_pgtos_atrasados'] )
df1['num_consultas_credito'] = np.where(df1['num_consultas_credito'] > 25, 4,df1['num_consultas_credito'] )
df1['taxa_juros'] = np.where(df1['taxa_juros'] > 40, 13,df1['taxa_juros'] )

In [28]:
# Idade                   - jovem 21, meia idade 35, maduro 59, idoso 60+
df1.idade = df1.idade.apply(lambda x: 
                            0 if x < 22 else 
                            2 if x < 36 else 
                            3 if x < 66 else
                            1)

# Saldo atual             - baixo 99, médio 199, alto 200+
df1.saldo_atual = df1.saldo_atual.apply(lambda x: 
                            0 if x < 10 else 
                            1 if x < 200 else                             
                            2)

# divida_atual            - baixo 499, médio 999, alto 1000+
df1.divida_atual = df1.divida_atual.apply(lambda x: 
                            0 if x > 1000 else 
                            1 if x > 499 else                             
                            2)

df1.renda_anual = df1.renda_anual.apply(lambda x: 
                            0 if x < 27000 else 
                            1 if x < 30000 else                             
                            2 if x < 50000 else
                            3 if x < 80000 else
                            4 if x < 120000 else
                            5 if x < 15000 else
                            6)

# valor_em_investimentos  - baixo 10, médio 250, alto 250+
df1.valor_em_investimentos = df1.valor_em_investimentos.apply(lambda x: 
                            0 if x < 50 else 
                            1 if x < 100 else                             
                            2)

# taxa_utilizacao_credito - baixo 25, médio 35, alto 35+
df1.taxa_utilizacao_credito = df1.taxa_utilizacao_credito.apply(lambda x: 
                            0 if x > 40 else 
                            1 if x > 25 else                             
                            2)

# num_emprestimos         - baixo 0, médio 3, alto 3+
df1.num_emprestimos = df1.num_emprestimos.apply(lambda x: 
                            0 if x > 3 else 
                            1 if x > 1 else                             
                            2)

# num_contas_bancarias    - baixo 0, médio 6, alto 6+
df1.num_contas_bancarias = df1.num_contas_bancarias.apply(lambda x: 
                            0 if x < 1 else 
                            1 if x < 7 else                             
                            2)

# num_cartoes_credito     - baixo 2, médio 5, alto 5+
df1.num_cartoes_credito = df1.num_cartoes_credito.apply(lambda x: 
                            0 if x < 2 else 
                            1 if x < 6 else                             
                            2)

# dias_atraso_dt_venc     - baixo 0, médio 10, alto 10+
df1.dias_atraso_dt_venc = df1.dias_atraso_dt_venc.apply(lambda x: 
                            0 if x > 10 else 
                            1 if x > 1 else                             
                            2)

# num_pgtos_atrasados     - baixo 3, médio 10, alto 10+
df1.num_pgtos_atrasados = df1.num_pgtos_atrasados.apply(lambda x: 
                            0 if x > 10 else 
                            1 if x > 3 else                             
                            2)

# num_consultas_credito   - baixo 2, médio 8, alto 8+
df1.num_consultas_credito = df1.num_consultas_credito.apply(lambda x: 
                            0 if x < 2 else 
                            1 if x < 8 else                             
                            2)

# taxa_juros              - baixo 5, médio 10, alto 10+
df1.taxa_juros = df1.taxa_juros.apply(lambda x: 
                            0 if x > 12 else 
                            1 if x > 4 else                             
                            2)



In [29]:
df1.describe().T

,count,mean,std,min,25%,50%,75%,max
id_cliente,9500.00,6210.53,3603.24,1.00,3091.75,6172.50,9320.25,12500.00
idade,9500.00,2.43,0.78,0.00,2.00,3.00,3.00,3.00
saldo_atual,9500.00,1.92,0.27,0.00,2.00,2.00,2.00,2.00
divida_atual,9500.00,0.64,0.81,0.00,0.00,0.00,1.00,2.00
renda_anual,9500.00,2.34,1.86,0.00,0.00,2.00,4.00,6.00
valor_em_investimentos,9500.00,1.52,0.71,0.00,1.00,2.00,2.00,2.00
taxa_utilizacao_credito,9500.00,1.03,0.38,0.00,1.00,1.00,1.00,2.00
num_emprestimos,9500.00,0.77,0.78,0.00,0.00,1.00,1.00,2.00
num_contas_bancarias,9500.00,1.32,0.55,0.00,1.00,1.00,2.00,2.00
num_cartoes_credito,9500.00,1.46,0.54,0.00,1.00,1.00,2.00,2.00


In [30]:

# Fillout NA
# Não é necessário

# Change Types
# Estatística descritiva básica 
# Feature engeneering

# Filtragem de variáveis inúteis
df1.drop(columns=['investe_exterior'], inplace=True)
df1.drop(columns=['pessoa_polit_exp'], inplace=True)



df2 = df1.copy()

# Saldo devedor por dia (divida_atual / dias_atraso_dt_venc)
df2['divida_diaria'] = df2.apply(lambda x: x['divida_atual'] / x['dias_atraso_dt_venc'] if x['dias_atraso_dt_venc'] > 0 else 0, axis=1)

# Colocar dias de atraso menor que zero ser igual a 0.
df2['dias_atraso_dt_venc'] = df2['dias_atraso_dt_venc'].apply(lambda x: 0 if x < 0 else x)

# Liquidez (saldo_atual menos divida_atual)
# df2['liquidez'] = df2.apply(lambda x: 0 if x['saldo_atual'] == 0 else (x['divida_atual'] / x['saldo_atual']) * 100, axis=1)

# Esforço (divida atual / renda anual) Em percentual
# df2['esforco'] = df2.apply(lambda x: 0 if x['renda_anual'] == 0 else (x['divida_atual'] / x['renda_anual']) * 100, axis=1)

df = df2.copy()

In [31]:
# Dividir dataset entre Treino (80%) e Teste (20%) 
X = df.drop('limite_adicional', axis = 1).copy()
y = df['limite_adicional'].copy()

x_train, x_test, y_train, y_test = ms.train_test_split(X, y, stratify = y, test_size = 0.20, random_state = 42)

In [32]:
# df2 será usado para splitar Treino e Validação
df2 = pd.concat([x_train, y_train], axis = 1)

# Dividir dataset entre Treino (90%) e Validação (10%)
X = df2.drop(['limite_adicional'], axis=1).copy()
y = df2['limite_adicional'].copy()

x_train, x_val, y_train, y_val = ms.train_test_split(X, y, stratify = y, test_size=0.10, random_state = 42)

In [33]:
# Dados de treino
print('x_train -> ',x_train.shape)
print('y_train -> ',y_train.shape)
# Dados de validação
print('x_val -> ',x_val.shape)
print('y_val -> ',y_val.shape)
# Dados de teste
print('x_test -> ',x_test.shape)
print('y_test -> ',y_test.shape)

x_train ->  (6840, 15)
y_train ->  (6840,)
x_val ->  (760, 15)
y_val ->  (760,)
x_test ->  (1900, 15)
y_test ->  (1900,)


# Modificar para testar

In [34]:
# Preparação dos dados
# Encodar features categóricas de treino
le_inv_ext = LabelEncoder()
le_pes_pol = LabelEncoder()
# x_train['investe_exterior'] = le_inv_ext.fit_transform(x_train['investe_exterior'])
# x_train['pessoa_polit_exp'] = le_pes_pol.fit_transform(x_train['pessoa_polit_exp'])

# Encodar var resposta de treino
lim_adi_dict = {'Negar':0, 'Conceder':1}
y_train = y_train.map(lim_adi_dict)

# Encodar features categóricas de validação
# x_val['investe_exterior']   = le_inv_ext.transform(x_val['investe_exterior'])
# x_val['pessoa_polit_exp']   = le_pes_pol.transform(x_val['pessoa_polit_exp'])

# Encodar var resposta de validação
y_val = y_val.map(lim_adi_dict)

In [35]:
x_train.describe().T

,count,mean,std,min,25%,50%,75%,max
id_cliente,6840.00,6220.06,3596.88,1.00,3102.75,6209.50,9315.25,12499.00
idade,6840.00,2.44,0.77,0.00,2.00,3.00,3.00,3.00
saldo_atual,6840.00,1.92,0.27,0.00,2.00,2.00,2.00,2.00
divida_atual,6840.00,0.64,0.81,0.00,0.00,0.00,1.00,2.00
renda_anual,6840.00,2.33,1.86,0.00,0.00,2.00,4.00,6.00
valor_em_investimentos,6840.00,1.52,0.71,0.00,1.00,2.00,2.00,2.00
taxa_utilizacao_credito,6840.00,1.03,0.37,0.00,1.00,1.00,1.00,2.00
num_emprestimos,6840.00,0.78,0.78,0.00,0.00,1.00,1.00,2.00
num_contas_bancarias,6840.00,1.32,0.55,0.00,1.00,1.00,2.00,2.00
num_cartoes_credito,6840.00,1.45,0.54,0.00,1.00,1.00,2.00,2.00


In [36]:
x_train.sample(10)

,id_cliente,idade,saldo_atual,divida_atual,renda_anual,valor_em_investimentos,taxa_utilizacao_credito,num_emprestimos,num_contas_bancarias,num_cartoes_credito,dias_atraso_dt_venc,num_pgtos_atrasados,num_consultas_credito,taxa_juros,divida_diaria
6367,9979,2,2,1,4,2,1,2,1,2,1,2,1,1,1.00
9172,5930,2,2,2,6,2,1,2,1,1,0,0,0,1,0.00
9140,1382,3,2,2,4,2,1,0,0,0,1,2,0,2,2.00
788,5329,3,2,0,4,2,1,0,2,1,0,0,1,0,0.00
7169,3365,3,2,0,0,0,1,0,1,2,0,0,2,0,0.00
7603,628,3,2,1,4,2,1,1,1,1,0,1,0,2,0.00
6248,11810,3,2,0,2,2,1,1,1,1,0,0,1,0,0.00
7395,3908,0,2,0,2,1,1,0,2,2,0,0,1,0,0.00
4208,7503,3,2,1,2,1,1,2,1,1,1,1,0,2,1.00
5735,8546,3,2,0,6,2,0,1,1,0,0,0,0,1,0.00


In [37]:
# Aplicação de modelos 
# Treinar XGBoost Classifier
# Model definition
model_xgboost = XGBClassifier(random_state=42)
# Model training
model_xgboost.fit(x_train, y_train)
# Model prediction
yhat_xgboost = model_xgboost.predict(x_val)

In [38]:
# Avaliação de modelos 
# Validar modelo com F1 Score (dados validação)
print(f"XGBoost - Média ponderada com F1 Score (dados validação): {round(f1_score(y_val, yhat_xgboost, average='weighted') ,3)}") #binary=default

XGBoost - Média ponderada com F1 Score (dados validação): 0.87


In [39]:
# Validar modelo com F1 Score, no método cross validation (dados validação) 
print(f"K-folds: {cross_val_score(model_xgboost, x_val, y_val, scoring='f1', cv = 10)}") #'F1' métrica F1 Score p/ classificação binary
print(f"XGBoost - Média ponderada com F1 Score (dados validação): {round(cross_val_score(model_xgboost, x_val, y_val, scoring='f1_weighted', cv = 10).mean() ,3)}")
# Performance manteve-se próxima com cross validation

K-folds: [0.66666667 0.46153846 0.33333333 0.25       0.57142857 0.31578947
 0.5        0.57142857 0.45454545 0.31578947]
XGBoost - Média ponderada com F1 Score (dados validação): 0.841


In [40]:
# Testar modelos
# Antes, juntar dados de treino + validação, formando um novo dataset de treino.
x_train_full = pd.concat([x_train, x_val])
y_train_full = pd.concat([y_train, y_val])

In [41]:
# Encodar features categóricas de teste
# x_test['investe_exterior']   = le_inv_ext.transform(x_test['investe_exterior'])
# x_test['pessoa_polit_exp']   = le_pes_pol.transform(x_test['pessoa_polit_exp'])

# Encodar var resposta de teste
y_test = y_test.map(lim_adi_dict)

In [42]:
# Treinar nova XGBoost, com o novo dataset treino
# Model definition
model_xgboost_full = XGBClassifier(random_state=42)
# Model training
model_xgboost_full.fit(x_train_full, y_train_full)
# Model prediction
# Passar os dados de teste separados anteriormente
yhat_xgboost_full = model_xgboost_full.predict(x_test)

In [43]:
# Validar modelo com F1 Score (dados teste)
print(f"XGBoost - Média ponderada com F1 Score (dados teste): {round(f1_score(y_test, yhat_xgboost_full, average='weighted') ,3)}") #binary=default

XGBoost - Média ponderada com F1 Score (dados teste): 0.86


In [46]:
x_train.describe().T

,count,mean,std,min,25%,50%,75%,max
id_cliente,6840.00,6220.06,3596.88,1.00,3102.75,6209.50,9315.25,12499.00
idade,6840.00,2.44,0.77,0.00,2.00,3.00,3.00,3.00
saldo_atual,6840.00,1.92,0.27,0.00,2.00,2.00,2.00,2.00
divida_atual,6840.00,0.64,0.81,0.00,0.00,0.00,1.00,2.00
renda_anual,6840.00,2.33,1.86,0.00,0.00,2.00,4.00,6.00
valor_em_investimentos,6840.00,1.52,0.71,0.00,1.00,2.00,2.00,2.00
taxa_utilizacao_credito,6840.00,1.03,0.37,0.00,1.00,1.00,1.00,2.00
num_emprestimos,6840.00,0.78,0.78,0.00,0.00,1.00,1.00,2.00
num_contas_bancarias,6840.00,1.32,0.55,0.00,1.00,1.00,2.00,2.00
num_cartoes_credito,6840.00,1.45,0.54,0.00,1.00,1.00,2.00,2.00


In [44]:
print(erro!!!!!!)

SyntaxError: ignored

In [ ]:
# Resultados:
 # XGBoost - Média ponderada com F1 Score (dados validação): 0.86
 # XGBoost - Média ponderada com F1 Score (dados teste): 0.86
# O modelo apresenta boa capacidade de generalização (classificar dados inéditos). 

In [ ]:
# Carregar dataset teste
x_ktest = df_test.copy()
# Conferir dataset teste kaggle
x_ktest

In [ ]:
# Encodar features categóricas de teste kaggle
# x_ktest['investe_exterior'] = le_inv_ext.transform(x_ktest['investe_exterior'])
# x_ktest['pessoa_polit_exp'] = le_pes_pol.transform(x_ktest['pessoa_polit_exp'])
x_ktest.drop(columns=['investe_exterior', 'pessoa_polit_exp'], inplace=True)

In [ ]:
x_ktest.columns

In [ ]:
# Fazer a previsão do score usando o modelo final treinado:
yhat_xgboost_ktest = model_xgboost_full.predict(x_ktest)

In [ ]:
# Preparação para submissão
# Criar dataset final
df_sub = pd.concat([df_test['id_cliente'], pd.Series(yhat_xgboost_ktest, name='limite_adicional')], axis=1)

In [ ]:
# Traduzir var resposta
lim_adi_dic_final = {0 : 'Negar', 1: 'Conceder'}
df_sub['limite_adicional'] = df_sub['limite_adicional'].map( lim_adi_dic_final )

In [ ]:
# Visualizar arquivo final para submissão ao Kaggle
df_sub[:5]

In [ ]:
# Gerar export para submission no Kaggle
df_sub.to_csv('baseline_kaggle.csv', index=False) 